In [ ]:
import pandas as pd
from db_utils import get_engine

In [ ]:
dataset_path = r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset"

In [ ]:
customers = pd.read_csv(dataset_path + '/customers.csv')
order_items = pd.read_csv(dataset_path +'/order_items.csv')
order_payments = pd.read_csv(dataset_path +'/order_payments.csv')
order_reviews = pd.read_csv(dataset_path +'/order_reviews.csv')
orders = pd.read_csv(dataset_path +'/orders.csv')
products = pd.read_csv(dataset_path +'/products.csv')
sellers = pd.read_csv(dataset_path +'/sellers.csv')
data = {'customers': customers,
        'order_items': order_items, 
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'orders': orders,
        'products': products,
        'sellers': sellers}

In [ ]:
order_items.info()

Fact order items table.

In [ ]:
# Get the most important columns and also the calculated columns for the fact_order_items. Merge with orders to get the customer id we need for a query later.

fact_order_items = order_items[['order_id', 'order_item_id', 'product_id', 'seller_id', 'price', 'freight_value',
                                'total_price', 'delivery_time', 'total_payments_count', 'profit_margin']].merge(orders[['order_id', 'customer_id']], on='order_id', how='left')

        Customers Dimension

In [ ]:
# Get the dimension columns for customers table
dim_customers = customers[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
                           'customer_city', 'customer_state']]

Product Dimension

In [ ]:
# Get the dimension columns for products table
dim_products = products[['product_id', 'product_category_name',
                         'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm',
                         'product_width_cm']]

Sellers Dimension

In [ ]:
# Get the dimension columns for sellers table
dim_sellers = sellers[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']]

Date Dimension

In [ ]:
# Use order_purchase_timestamp and also order_delivered_customer_date for the date table. 
# First we want to set the purchase and delivery to datetime.  
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
dim_date = orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']].copy()

In [ ]:
engine = get_engine()

In [ ]:
# Fact Table
fact_order_items.to_sql('fact_order_items', con=engine, if_exists='replace', index=False)

# Dimension Tables
dim_customers.to_sql('dim_customers', con=engine, if_exists='replace', index=False)
dim_products.to_sql('dim_products', con=engine, if_exists='replace', index=False)
dim_sellers.to_sql('dim_sellers', con=engine, if_exists='replace', index=False)
dim_date.to_sql('dim_date', con=engine, if_exists='replace', index=False)